In [94]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, Input, metrics, Model
import matplotlib.pyplot as plt

from pandas import read_csv
from sklearn.model_selection import train_test_split

In [78]:
## import dataset
def load_file(filepath):
	dataframe = read_csv(filepath, header=None)
	return dataframe.values

In [79]:

X = load_file('datasetin.csv')
y = load_file('datasetout.csv')
trainX, testX, trainy, testy = train_test_split(X, y, random_state = 42, test_size = 0.2)


In [80]:
X.shape

(252, 10001)

In [101]:
inputs = Input(shape=(10001,))

##separating bits and snr
stream = inputs[:, :-1]
snr = inputs[:, -1]

x = layers.Bidirectional(layers.GRU(
    64//2
))(stream[:, :, None])

x = layers.Concatenate()([
    x, snr[:, None]
])

x = layers.Dense(64, activation=layers.LeakyReLU())(x)
x = layers.Dense(32, activation=layers.LeakyReLU())(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)

In [99]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 10001)]      0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_15 (Sl (None, 10000)        0           input_12[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_17 (Sl (None, 10000, 1)     0           tf.__operators__.getitem_15[0][0]
__________________________________________________________________________________________________
tf.__operators__.getitem_16 (Sl (None,)              0           input_12[0][0]                   
____________________________________________________________________________________________

In [105]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=[metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall(), metrics.AUC()])

history = model.fit(trainX, trainy, epochs=100, 
                    validation_data=(testX, testy))

Epoch 1/100
7/7 [==============================] - 26s 3s/step - loss: 0.6563 - binary_accuracy: 0.6471 - precision_5: 0.6397 - recall_5: 0.7054 - auc: 0.7263 - val_loss: 0.6317 - val_binary_accuracy: 0.6863 - val_precision_5: 0.6571 - val_recall_5: 0.8519 - val_auc: 0.7585
Epoch 2/100
7/7 [==============================] - 21s 3s/step - loss: 0.6738 - binary_accuracy: 0.5892 - precision_5: 0.5617 - recall_5: 0.5280 - auc: 0.6545 - val_loss: 0.6303 - val_binary_accuracy: 0.7255 - val_precision_5: 0.7241 - val_recall_5: 0.7778 - val_auc: 0.7461
Epoch 3/100
7/7 [==============================] - 21s 3s/step - loss: 0.6566 - binary_accuracy: 0.5883 - precision_5: 0.5614 - recall_5: 0.7893 - auc: 0.7082 - val_loss: 0.6220 - val_binary_accuracy: 0.7255 - val_precision_5: 0.7097 - val_recall_5: 0.8148 - val_auc: 0.7485
Epoch 4/100
7/7 [==============================] - 21s 3s/step - loss: 0.6267 - binary_accuracy: 0.6654 - precision_5: 0.6616 - recall_5: 0.6686 - auc: 0.7444 - val_loss: 0.64

KeyboardInterrupt: 